# Mesh Flow

## Imports, Logs, Parameters

In [1]:
from src.stabilization import main, parser
import os
import os.path as osp

## Example Usage

In [2]:
args = parser.parse_args([
    "../data/small-shaky-5.avi",
    "result/offline/",
])
main(args)

[I 2020-01-07 18:11:02 meshflow] ../data/small-shaky-5.avi


write     : 100%|█████████████████████████████████████████████████| 210/210 [00:05<00:00, 39.58it/s]

[I 2020-01-07 18:11:11 meshflow] time elapsed (s): 9.08


## Improved Method

In [3]:
dir_path = '../data'

for name in os.listdir(dir_path):
    args = parser.parse_args([
        osp.join(dir_path, name),
        "result/offline-fast/",
    ])
    main(args)
    
    print()

[I 2020-01-07 17:33:24 meshflow] ../data/parallax.avi


write     : 100%|█████████████████████████████████████████████████| 525/525 [00:52<00:00, 10.05it/s]

[I 2020-01-07 17:35:31 meshflow] time elapsed (s): 127.25

[I 2020-01-07 17:35:31 meshflow] ../data/selfie.mp4



write     :  97%|███████████████████████████████████████████████▌ | 168/173 [00:30<00:00,  5.56it/s]

[I 2020-01-07 17:36:19 meshflow] time elapsed (s): 48.18


...

[I 2020-01-07 17:42:35 meshflow] time elapsed (s): 102.11

[I 2020-01-07 17:42:35 meshflow] ../data/simple.avi



write     : 100%|████████████████████████████████████████████████▊| 447/449 [01:10<00:00,  6.33it/s]

[I 2020-01-07 17:44:21 meshflow] time elapsed (s): 106.08

[I 2020-01-07 17:44:21 meshflow] ../data/sample.avi



write     : 100%|████████████████████████████████████████████████▊| 399/401 [01:05<00:00,  6.05it/s]

[I 2020-01-07 17:46:07 meshflow] time elapsed (s): 105.67



## Original Method

In [ ]:
dir_path = '../data'

for name in os.listdir(dir_path):
    args = parser.parse_args([
        osp.join(dir_path, name),
        "result/offline-slow/",
        "--slow"
    ])
    main(args)
    
    print()

[I 2020-01-07 17:25:38 meshflow] ../data/parallax.avi


read      :   2%|█                                                 | 11/525 [00:40<31:47,  3.71s/it]